In [1]:
import pickle
import json
import numpy as np
import networkx as nx
import tqdm, os
from collections import Counter, defaultdict
from torchnlp.encoders.text import StaticTokenizerEncoder
import osmnx as ox
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = '6'

In [2]:
import yaml
'''修改这里的路径加载配置好的cfg文件'''
config_filename = f'../models/cfg.yaml'
configs = yaml.load(open(config_filename), Loader=yaml.FullLoader)

##### 获得节点id-token对应关系

In [3]:
G = pickle.load(open(configs['data_path'] + "longhua_1.8k.pkl", "rb"))
nodeid2token = StaticTokenizerEncoder(sample=[sorted(list(G.nodes()))],
                                      tokenize=lambda s: list(map(str, s)))
G = nx.MultiDiGraph(G)

In [4]:
len(G.nodes())

1839

##### 生成结果展示

In [5]:
def res_data(file='1m_50_tfm_bs'):
    data = configs['save_path'] + f'{file}.pkl'
    with open(data, 'rb') as f:
        data = pickle.load(f)
    if file.endswith("gnn"):
        data_cams = data['cam_id'][:, 0].tolist()
    else:
        if not isinstance(data['cam_id'], dict):
            data_cams = data['cam_id'].tolist()
        else:
            data_cams = list(data['cam_id'].values())
    if not isinstance(data['pred'], list):
        data_pred = data['pred'].tolist()
        data_grth = data['grth'].tolist()
    else:
        data_pred = data['pred']
        data_grth = data['grth']
    return data_pred, data_cams, data_grth

In [6]:
def res_data2(file='../hmm_rec_3D/hmm_rec.pkl'):
    data_file = '../dataset/'
    data = pickle.load(open(data_file + "data_sim_4k.pkl", "rb"))
    data_pred = pickle.load(open(file, 'rb'))
    for k in data_pred.keys():
        data_pred[k] = [data_pred[k][0][i][0] for i in range(len(data_pred[k][0]))]
    data_pred = list(data_pred.values())
    data_cams = list(data["cam_x"].values())[-50:][:len(data_pred)]
    data_grth = list(data['traj_y'].values())[-50:][:len(data_pred)]
    return data_pred, data_cams, data_grth

In [7]:
data_res = [
    'bs_shortest_path_3d',
    '../hmm_rec_3D/hmm_rec.pkl',
    "bs_shorest_path_deno",
    'bs_shorest_path_tklet_deno',
    'real_test_tfm_tklet_deno_3d',
]

nums = min([len(res_data(file)[0]) for file in data_res[:1]])
percision = [[] for _ in data_res]
recall = [[] for _ in data_res]
iou = [[] for _ in data_res]
per_wono = [[] for _ in data_res]
rec_wono = [[] for _ in data_res]
iou_wono = [[] for _ in data_res]
per_wino = [[] for _ in data_res]
rec_wino = [[] for _ in data_res]
iou_wino = [[] for _ in data_res]
wino_cid = []
wono_cid = []
cam_x = defaultdict(list)
res_traj = defaultdict(list)
grth_traj = defaultdict(list)
itrns = defaultdict(list)
unins = defaultdict(list)
for i in tqdm(range(nums)):
    for file_idx, file in enumerate(data_res):
        if not '/' in file:
            data_pred, data_cams, data_grth = res_data(file)
            vid = i

            od_clip = nodeid2token.decode(data_cams[vid])
            od_clip = list(map(int,
                            [i for i in od_clip.split(" ") if i[0] != '<']))

            ans = nodeid2token.decode(data_grth[vid])
            gt_traj = list(map(int, [i for i in ans.split(" ") if i[0] != '<']))
            if file_idx == 0:
                ans = od_clip
                gen_traj = ans
            else:
                ans = nodeid2token.decode(data_pred[vid])
                gen_traj = list(map(int, [i for i in ans.split(" ") if i[0] != '<']))
            # print(gen_traj)
            gen_traj_refine = [] if len(set(gen_traj)) > 1 else ox.shortest_path(
                G, gt_traj[0], gt_traj[1], weight='length')
            if len(set(gen_traj)) > 1:
                # paths = None
                for j in range(len(gen_traj) - 1):
                    paths = ox.shortest_path(G,
                                            gen_traj[j],
                                            gen_traj[j + 1],
                                            weight='length')
                    if not paths:
                        break
                    if od_clip[-1] in paths:
                        ix = len(paths) - paths[::-1].index(od_clip[-1])
                        gen_traj_refine.extend(paths[:ix])
                        break
                    if j == len(gen_traj) - 2:
                        gen_traj_refine.extend(paths)
                    else:
                        gen_traj_refine.extend(paths[:-1])
            # lens = len(gen_traj_refine)
            # tmp = gen_traj_refine[:]
            for n in gen_traj_refine[::-1]:
                if n in od_clip:
                    break
                gen_traj_refine.pop(-1)
            res_traj[file_idx].append(gen_traj_refine)
            grth_traj[file_idx].append(gt_traj)
            cam_x[file_idx].append(od_clip)
            # if 'shortest_path' in data_res[file_idx]:
            #     gen_traj_refine = gen_traj
            ints = set(gen_traj_refine) & set(gt_traj)
            unin = set(gen_traj_refine) | set(gt_traj)
            itrns[file_idx].append(ints)
            unins[file_idx].append(unin)
            if len(set(od_clip) - (set(gt_traj) & set(od_clip))) < 3:
                per_wono[file_idx].append(
                    len(ints) * 1. / (len(set(gen_traj_refine)) + 1e-8))
                iou_wono[file_idx].append(len(ints) * 1. / len(set(unin)))
                rec_wono[file_idx].append(len(ints) * 1. / len(set(gt_traj)))
                if file_idx == 2:
                    wono_cid.append(vid)
            else:

                per_wino[file_idx].append(
                    len(ints) * 1. / (len(set(gen_traj_refine)) + 1e-8))
                iou_wino[file_idx].append(len(ints) * 1. / len(set(unin)))
                rec_wino[file_idx].append(
                    len(ints) * 1. / (len(set(gt_traj)) + 1e-8))
                if file_idx == 2:
                    wino_cid.append(vid)
            percision[file_idx].append(
                len(ints) * 1. / (len(set(gen_traj_refine)) + 1e-8))
            iou[file_idx].append(len(ints) * 1. / len(set(unin)))
            recall[file_idx].append(len(ints) * 1. / (len(set(gt_traj)) + 1e-8))
        else:
            data_pred, data_cams, data_grth = res_data2(file)
            vid = i
            # if vid == 19:
            #     continue
            od_clip = data_cams[vid]
            ans = data_grth[vid]
            gt_traj = data_grth[vid]

            ans = data_pred[vid]
            gen_traj = ans
            # print(gen_traj)
            # print(gt_traj)
            gen_traj_refine = [] if len(set(gen_traj)) > 1 else nx.shortest_path(
                G, gt_traj[0], gt_traj[1], weight='length')
            if len(set(gen_traj)) > 1:
                # paths = None
                for j in range(len(gen_traj) - 1):
                    paths = ox.shortest_path(G,
                                            gen_traj[j],
                                            gen_traj[j + 1],
                                            weight='length')
                    if not paths:
                        break
                    if od_clip[-1] in paths:
                        ix = len(paths) - paths[::-1].index(od_clip[-1])
                        gen_traj_refine.extend(paths[:ix])
                        break
                    if j == len(gen_traj) - 2:
                        gen_traj_refine.extend(paths)
                    else:
                        gen_traj_refine.extend(paths[:-1])
            gen_o = 0
            gen_d = len(gen_traj_refine)
            if od_clip[0] in gen_traj_refine:
                gen_o = gen_traj_refine.index(od_clip[0])
            if od_clip[-1] in gen_traj_refine:
                gen_d = len(gen_traj_refine) - gen_traj_refine[::-1].index(
                    od_clip[-1])
            if gen_d > gen_o:
                gen_traj_refine = gen_traj_refine[gen_o:gen_d]
            res_traj[file_idx].append(gen_traj_refine)
            grth_traj[file_idx].append(gt_traj)
            cam_x[file_idx].append(od_clip)
            # if 'shortest_path' in data_res[file_idx]:
            #     gen_traj_refine = gen_traj
            ints = set(gen_traj_refine) & set(gt_traj)
            unin = set(gen_traj_refine) | set(gt_traj)
            itrns[file_idx].append(ints)
            unins[file_idx].append(unin)
            if len(set(od_clip) - (set(gt_traj) & set(od_clip))) < 1:
                per_wono[file_idx].append(
                    len(ints) * 1. / (len(set(gen_traj_refine)) + 1e-8))
                iou_wono[file_idx].append(len(ints) * 1. / len(set(unin)))
                rec_wono[file_idx].append(len(ints) * 1. / len(set(gt_traj)))
                if file_idx == 1:
                    wono_cid.append(vid)
            else:

                per_wino[file_idx].append(
                    len(ints) * 1. / (len(set(gen_traj_refine)) + 1e-8))
                iou_wino[file_idx].append(len(ints) * 1. / len(set(unin)))
                rec_wino[file_idx].append(
                    len(ints) * 1. / (len(set(gt_traj)) + 1e-8))
                if file_idx == 1:
                    wino_cid.append(vid)
            percision[file_idx].append(
                len(ints) * 1. / (len(set(gen_traj_refine)) + 1e-8))
            iou[file_idx].append(len(ints) * 1. / len(set(unin)))
            recall[file_idx].append(len(ints) * 1. / (len(set(gt_traj)) + 1e-8))
# pickle.dump(
#     {
#         'pred': res_traj,
#         'grth': grth_traj,
#         'cam_x': cam_x,
#     }, open(configs['save_path'] + 'vi_traj.pkl', 'wb'))


 56%|█████▌    | 28/50 [00:00<00:00, 89.51it/s]

100%|██████████| 50/50 [00:00<00:00, 87.23it/s]


In [8]:
print([sum(percision[i]) / len(percision[i]) for i in range(len(data_res))])
print([sum(recall[i]) / len(recall[i]) for i in range(len(data_res))])
print([sum(iou[i]) / len(iou[i]) for i in range(len(data_res))])

[0.8606186069270081, 0.8641051763623842, 0.9017083339581204, 0.7998679476383014, 0.8957872620386831]
[0.9197220675593911, 0.8996063891395631, 0.8912515162040553, 0.9134072021130782, 0.9235442721613996]
[0.8242757222106295, 0.8106001587811753, 0.8385775760587992, 0.7568854570767427, 0.851813318742058]
